In [19]:
import cv2
import json
import tensorflow as tf
import tensorflow_hub as hub

# Load MoveNet model
model = hub.load('https://tfhub.dev/google/movenet/singlepose/lightning/4')

# Define keypoint names based on MoveNet's known order
KEYPOINT_NAMES = [
    "nose", "left_eye", "right_eye", "left_ear", "right_ear", 
    "left_shoulder", "right_shoulder", "left_elbow", "right_elbow",
    "left_wrist", "right_wrist", "left_hip", "right_hip",
    "left_knee", "right_knee", "left_ankle", "right_ankle"
]

def movenet(input_image):
    input_image = tf.image.resize_with_pad(tf.expand_dims(input_image, axis=0), 192, 192)
    input_image = tf.cast(input_image, dtype=tf.int32)
    outputs = model.signatures['serving_default'](input_image)
    keypoints = outputs['output_0'].numpy()
    return keypoints

def serialize_landmarks(keypoints):
    serialized = []
    for idx, kp in enumerate(keypoints[0, 0, :]):
        serialized.append({
            'name': KEYPOINT_NAMES[idx],
            'x': float(kp[1]),
            'y': float(kp[0]),
            'score': float(kp[2])
        })
    return serialized

# Load video
video = cv2.VideoCapture('demo_video.mp4')
demo_poses = []

while video.isOpened():
    ret, frame = video.read()
    if not ret:
        break

    # Convert the frame to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Process the frame to extract pose
    keypoints = movenet(rgb_frame)
    serialized_landmarks = serialize_landmarks(keypoints)
    demo_poses.append(serialized_landmarks)

video.release()

# Save the demo poses for future comparison
with open('demo_poses.json', 'w') as f:
    json.dump({'poses': demo_poses}, f, indent=4)


In [28]:
import numpy as np
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

angle = calculate_angle([ 0.4246756903701565, 0.3882735731072927 ], [ 0.45501756482551065, 0.4028621332190844 ], [ 0.3768017245637768, 0.27402184631110166 ])
print(angle)

33.06053490331318


In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# Load the MoveNet model from TensorFlow Hub
model = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")

# Function to process the image and prepare it for the model
def preprocess_image(image_path, input_size=192):
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"File {image_path} does not exist.")
    
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Failed to read the image {image_path}.")
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (input_size, input_size))
    img = img.astype(np.int32)  # Convert to int32
    img = np.expand_dims(img, axis=0)
    return img

# Function to detect poses in the image
def detect_pose(model, img):
    img_tensor = tf.convert_to_tensor(img, dtype=tf.int32)  # Ensure the tensor is int32
    outputs = model.signatures["serving_default"](img_tensor)
    keypoints = outputs['output_0'].numpy()
    return keypoints

# Function to draw keypoints and edges on the image
def draw_keypoints(image, keypoints, confidence_threshold=0.1):
    y, x, _ = image.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))

    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(image, (int(kx), int(ky)), 4, (0, 255, 0), -1)
    return image

# Path to the image
image_path = 'frozen-shoulder-exercise/stage-2.png'

# Preprocess the image
processed_image = preprocess_image(image_path)

# Detect the pose
keypoints = detect_pose(model, processed_image)

body_parts = [
    "nose", "left_eye", "right_eye", "left_ear", "right_ear", "left_shoulder", "right_shoulder",
    "left_elbow", "right_elbow", "left_wrist", "right_wrist", "left_hip", "right_hip",
    "left_knee", "right_knee", "left_ankle", "right_ankle"
]

# Extract the keypoints for each body part and create the required dictionary format
formatted_keypoints = []
for i, part in enumerate(body_parts):
    kp = keypoints[0, 0, i]
    formatted_keypoints.append({
        "name": part,
        "x": kp[0],
        "y": kp[1],
        "score": kp[2]
    })

# Print the formatted keypoints
print(formatted_keypoints)

[{'name': 'nose', 'x': 0.3809781, 'y': 0.53151387, 'score': 0.87499297}, {'name': 'left_eye', 'x': 0.35043752, 'y': 0.562536, 'score': 0.56667644}, {'name': 'right_eye', 'x': 0.34894747, 'y': 0.5005225, 'score': 0.62537974}, {'name': 'left_ear', 'x': 0.3679764, 'y': 0.6071831, 'score': 0.6449508}, {'name': 'right_ear', 'x': 0.367179, 'y': 0.4601211, 'score': 0.6620657}, {'name': 'left_shoulder', 'x': 0.45170206, 'y': 0.67959785, 'score': 0.739439}, {'name': 'right_shoulder', 'x': 0.44737315, 'y': 0.39777395, 'score': 0.87130123}, {'name': 'left_elbow', 'x': 0.29085112, 'y': 0.70728415, 'score': 0.8915762}, {'name': 'right_elbow', 'x': 0.2888388, 'y': 0.35413033, 'score': 0.6532154}, {'name': 'left_wrist', 'x': 0.12878864, 'y': 0.5653012, 'score': 0.71840924}, {'name': 'right_wrist', 'x': 0.13008459, 'y': 0.48835444, 'score': 0.6074091}, {'name': 'left_hip', 'x': 0.7775527, 'y': 0.6293003, 'score': 0.7674307}, {'name': 'right_hip', 'x': 0.77141535, 'y': 0.41100162, 'score': 0.77271837},